In [96]:
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import corner
import json

import pytensor
from pytensor import tensor as pt
from pytensor.graph.basic import Apply
from pytensor.graph import Apply, Op

import jax
from jax import grad, jit, vmap, value_and_grad, random
import jax.numpy as jnp

import pyhf
pyhf.set_backend('jax')

import pymc as pm
import arviz as az

from pyhf_pymc import prepare_inference
from pyhf_pymc import prepare_inference_combined
from pyhf_pymc import infer
from pyhf_pymc import make_op
from pyhf_pymc import plotting

%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
blue = '#1F449C'
pink = '#E57A77'

In [94]:
## Simple pyhf model
model = pyhf.Model(
    {'channels': [{'name': 'singlechannel',
    'samples': [
    {'name': 'signal',
     'data': [6, 6, 10],
     'modifiers': [
         {'name': 'mu', 'type': 'normfactor', 'data': None}
        ]},

    {'name': 'background',
     'data': [550, 550, 550]
     ,
     'modifiers': [
        ## Correlated / Normal
        {'name': 'corr_bkg', 'type': 'histosys','data': {'hi_data': [650, 560, 670], 'lo_data': [40, 40, 43]}},
        {'name': 'corr_bkg1', 'type': 'histosys','data': {'hi_data': [650, 560, 670], 'lo_data': [300, 350, 400]}},
        # ## Uncorrelated / Poisson
        {'name': 'uncorr_bkg', 'type': 'shapesys','data': [7, 8, 7.17]},
        {'name': 'uncorr_bkg1', 'type': 'shapesys','data': [7, 8, 7.17]},
        # {'name': 'uncorr_bkg2', 'type': 'shapesys','data': [8, 9, 10]},
         ]
         },    
                                 
    ]},
    ]
        }
)

data = model.expected_actualdata(model.config.suggested_init())

ur_hyperparameters = {}
ur_hyperparameters['Gamma'] = [[1., 0.1], [1., 0.1], [1., 0.1]]
ur_hyperparameters['Normal'] = [[0, 2], [0, 2], [0, 2]]

In [89]:
prepare_inference.build_priorDict_conjugate(model, unconstr_priors, ur_hyperparameters)

{'corr_bkg': {'type': 'Normal',
  'mu': array([0.]),
  'sigma': array([0.89442719])},
 'corr_bkg1': {'type': 'Normal',
  'mu': array([0.]),
  'sigma': array([0.89442719])},
 'mu': {'type': 'HalfNormal_Unconstrained', 'sigma': [0.4]},
 'uncorr_bkg': {'type': 'Gamma',
  'alpha': array([6174.46938776, 4727.5625    , 5885.19514909]),
  'beta': array([6790.81632653, 5199.21875   , 6472.614664  ])},
 'uncorr_bkg1': {'type': 'Gamma',
  'alpha': array([6174.46938776, 4727.5625    , 5885.19514909]),
  'beta': array([6790.81632653, 5199.21875   , 6472.614664  ])}}

In [92]:
prepare_inference.build_priorDict_conjugate(model, unconstr_priors)

{'corr_bkg': {'type': 'Normal',
  'mu': array([20.]),
  'sigma': array([0.89442719])},
 'corr_bkg1': {'type': 'Normal',
  'mu': array([20.]),
  'sigma': array([0.89442719])},
 'mu': {'type': 'HalfNormal_Unconstrained', 'sigma': [0.4]},
 'uncorr_bkg': {'type': 'Gamma',
  'alpha': array([6174.46938776, 4727.5625    , 5885.19514909]),
  'beta': array([6790.81632653, 5199.21875   , 6472.614664  ])},
 'uncorr_bkg1': {'type': 'Gamma',
  'alpha': array([6174.46938776, 4727.5625    , 5885.19514909]),
  'beta': array([6790.81632653, 5199.21875   , 6472.614664  ])}}

In [97]:
with infer.model_conjugate(model, unconstr_priors, data, ur_hyperparameters):
    step = pm.Metropolis()
    post_data_combined = pm.sample(draws=n_draws, chains=1, step=step)
    # post_pred_combined = pm.sample_posterior_predictive(post_data_combined)
    # prior_pred_combined = pm.sample_prior_predictive(10_000)

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [corr_bkg]
>Metropolis: [corr_bkg1]
>Metropolis: [mu]
>Metropolis: [uncorr_bkg]
>Metropolis: [uncorr_bkg1]


Sampling 1 chain for 1_000 tune and 5_000 draw iterations (1_000 + 5_000 draws total) took 6 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [98]:
with infer.model_conjugate(model, unconstr_priors, data):
    step = pm.Metropolis()
    post_data_combined = pm.sample(draws=n_draws, chains=1, step=step)
    # post_pred_combined = pm.sample_posterior_predictive(post_data_combined)
    # prior_pred_combined = pm.sample_prior_predictive(10_000)

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [corr_bkg]
>Metropolis: [corr_bkg1]
>Metropolis: [mu]
>Metropolis: [uncorr_bkg]
>Metropolis: [uncorr_bkg1]


Sampling 1 chain for 1_000 tune and 5_000 draw iterations (1_000 + 5_000 draws total) took 6 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [99]:
ur_hyperparameters

{'Gamma': [[1.0, 0.1], [1.0, 0.1], [1.0, 0.1]],
 'Normal': [[0, 2], [0, 2], [0, 2]]}